In [1]:
from utils.checkpoints import load_checkpoint
from constants import *
from models.densenet121 import DenseNet121_change_avg
from dataloader.dataloader import HmDataset
from dataloader.transforms import transforms
from main.fit import fit

import numpy as np
import pandas as pd
from PIL import Image

from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn

from tqdm import tqdm

In [2]:
class HmDataset(Dataset):
    def __init__(self, df_path, transforms=None, mode='3ch'):
        self.df = pd.read_csv(df_path)
        self.transforms = transforms
        self.mode = mode
        
    def __getitem__(self, index):
        hm_meta = self.df.iloc[index]
        filename = hm_meta.filename
        label = torch.from_numpy(hm_meta['epidural':'any'].values.astype(np.float))
        
        img = Image.open('../dataset/kaggle_rsna(only600)/imgs/'+filename+'.png')
        
        if self.mode=='sequential':
            study_instance_uid = hm_meta.study_instance_uid
            slice_id_current = hm_meta.slice_id
            
            # 해당 환자의 DataFrame
            tmp_df = self.df[self.df.study_instance_uid==study_instance_uid]
            
            # 해당 환자의 slice 전체 수 
            max_slice = tmp_df.shape[0]
            
            # get prev, next slice number
            slice_id_prev = slice_id_current if slice_id_current==0 else slice_id_current-1
            slice_id_next = slice_id_current if slice_id_current==max_slice-1 else slice_id_current+1
            
            # get prev, next filename
            filename_prev = tmp_df[tmp_df.slice_id==slice_id_prev].iloc[0].filename
            filename_next = tmp_df[tmp_df.slice_id==slice_id_next].iloc[0].filename
            
            # get prev, next img
            img_prev = Image.open('../dataset/kaggle_rsna(only600)/imgs/'+filename_prev+'.png')
            img_next = Image.open('../dataset/kaggle_rsna(only600)/imgs/'+filename_next+'.png')
            
            # concat 3 imgs
            img = np.concatenate([np.expand_dims(np.array(img)[:,:,0], axis=2),
                                   np.expand_dims(np.array(img_prev)[:,:,0], axis=2),
                                   np.expand_dims(np.array(img_next)[:,:,0], axis=2)],
                                  axis=2)
            img = Image.fromarray(img)
                                   

        if self.transforms is not None:
            img = self.transforms(img)
        
        return filename, label, img
    
    def __len__(self):
        return len(self.df)

train_dataset = HmDataset(df_path='./dataset/train.csv', transforms=transforms, mode=DATASET_MODE)
valid_dataset = HmDataset(df_path='./dataset/valid.csv', transforms=transforms, mode=DATASET_MODE)
test_dataset = HmDataset(df_path='./dataset/test.csv', transforms=transforms, mode=DATASET_MODE)

In [3]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset,
                         batch_size=1,
                         shuffle=True,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4)
test_loader = DataLoader(test_dataset,
                        batch_size=1,
                        shuffle=False,
                        num_workers=1)

In [4]:
# get test data loader
# test_dataset = HmDataset(df_path='./dataset/test.csv', transforms=transforms, mode=DATASET_MODE)
# test_loader = DataLoader(test_dataset,
#                          batch_size=1,#BATCH_SIZE,
#                          shuffle=False,
#                          drop_last=False,
#                          num_workers=4)

# create model
model = DenseNet121_change_avg()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=0.9)

model =  DenseNet121_change_avg()
model, _, _, epoch = load_checkpoint('./checkpoints/cnn/200615_135016_DenseNet121_LR0.001_BS64_BCELoss/030.pth', model, optimizer)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# test 평가

with torch.no_grad():
    
    for i, (filename, targets, inputs) in enumerate(tqdm(test_loader, position=0, leave=True)):
        inputs, targets = inputs.to(device), targets.to(device)
        
        preds, feas = model(inputs)
        
        
        

100%|██████████| 2073/2073 [01:16<00:00, 27.15it/s]


In [17]:
len(train_loader), len(test_loader)

(261, 2073)

In [6]:
for pp in p:
    print(pp.sum())

tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0., device='c